In [1]:
#I crawled the craigslist for couple of days to find the used Toyota car information
#Below is my analysis to figure out the average price and average price by year.

#The biggest question is between the correlation of price to odometer reading and
# to made-of-year, which one is stronger????

In [2]:
# File location and type
file_location = "/FileStore/tables/deals.jl"
file_type = "json"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df,5)

Price,Title,Url,odometer
$36988,2016 Toyota Highlander HYBRID LIMITED 4WD 3RD ROW ADAPTIVE CRUISE REAR,https://portland.craigslist.org/mlt/ctd/d/portland-2016-toyota-highlander-hybrid/6969229725.html,31745
$36988,2016 Toyota Highlander HYBRID LIMITED 4WD 3RD ROW ADAPTIVE CRUISE REAR,https://portland.craigslist.org/mlt/ctd/d/portland-2016-toyota-highlander-hybrid/6969229725.html,31745
$9995,2008 Toyota Highlander AWD SUV,https://portland.craigslist.org/mlt/ctd/d/portland-2008-toyota-highlander-awd-suv/6969309434.html,clean
$17000,2010 Toyota Highlander Hybrid Limited 52K miles!,https://portland.craigslist.org/clk/cto/d/vancouver-2010-toyota-highlander-hybrid/6969344007.html,hybrid
$8995,2011 Toyota Highlander,https://portland.craigslist.org/mlt/ctd/d/happy-valley-2011-toyota-highlander/6969344814.html,gas
$9995,2008 Toyota Highlander AWD SUV,https://portland.craigslist.org/mlt/ctd/d/portland-2008-toyota-highlander-awd-suv/6969309434.html,clean
$17000,2010 Toyota Highlander Hybrid Limited 52K miles!,https://portland.craigslist.org/clk/cto/d/vancouver-2010-toyota-highlander-hybrid/6969344007.html,hybrid
$8995,2011 Toyota Highlander,https://portland.craigslist.org/mlt/ctd/d/happy-valley-2011-toyota-highlander/6969344814.html,gas
$4995,2001 Toyota Highlander 4dr 4-Cyl 4WD*CLEAN TITLE*ONE OWNER*NEW TIRE*RUNS&DRIVE G,https://portland.craigslist.org/wsc/ctd/d/2001-toyota-highlander-4dr-4-cyl/6969351127.html,4wd
$4995,2001 Toyota Highlander 4dr 4-Cyl 4WD*CLEAN TITLE*ONE OWNER*NEW TIRE*RUNS&DRIVE G,https://portland.craigslist.org/wsc/ctd/d/2001-toyota-highlander-4dr-4-cyl/6969351127.html,4wd


In [3]:

table_name = "car_deals"
df.write.mode('OverWrite').format('parquet').saveAsTable(table_name)


In [4]:
from pyspark.sql.functions import avg, col, udf,desc

@udf('integer')
def convert_value(value):
  try:
    return int(value.replace('$',''))
  except ValueError:
    return None
  


In [5]:
df.count()

Out[ 5 ]: 603

In [6]:
#delete the duplicates

df=df.distinct()
df.count()

Out[ 6 ]: 493

In [7]:
#in case there are multiple prices for one car, let's get the average price for one car
one_car_avg_price= df\
  .select(convert_value('price').alias('price'),'Url')\
  .groupBy('Url')\
  .agg(avg('price').cast('integer'))


In [8]:
one_car_avg_price.show(5)

+--------------------+----------+
 Url|avg(price)|
+--------------------+----------+
https://portland....| 14995.0|
https://portland....| 4500.0|
https://portland....| 6800.0|
https://portland....| 5800.0|
https://portland....| 5250.0|
+--------------------+----------+
only showing top 5 rows

In [9]:
#average price of all cars
display(
  one_car_avg_price.select('avg(price)').agg(avg('avg(price)').cast('integer'))
)

CAST(avg(avg(price)) AS INT)
13163


In [10]:
import re

#extract the year information from Url
@udf('integer')
def get_year(value):
  try:
    year_search=re.search('-(\d+)-toyota-',value,re.IGNORECASE)
    if year_search:
      return int(year_search.group(1))
  except ValueError:
    return None

In [11]:
#Average price by year
display(
  df\
    .select(get_year('Url').alias('year'), convert_value('price').alias('price'),)\
    .where(col('year')>=1950 )\
    .groupBy('year')\
    .agg(avg('price').cast('integer'))\
    .sort('year')
)

year,CAST(avg(price) AS INT)
1967,22000
1977,750
1983,22500
1985,3175
1987,1000
1990,4300
1991,3995
1993,4200
1994,9000
1995,2100


In [12]:
#It seems not much of correlation between year and average price. Let's see whether there is a correlation
#between odometer reading and price 
@udf('integer')
def get_odo(value):
  try:
    odo_search=re.search('(\d+)',value,re.IGNORECASE)
    if odo_search:
      return int(odo_search.group(1))
    else: 0
  except ValueError:
    return None

In [13]:
display(
  df\
    .select(get_odo('odometer').alias('odometer'), convert_value('price').alias('price'),)\
    .where(col('odometer').isNotNull() & (col('odometer') >=1000))   
)

odometer,price
140534,16995
13666,31881
291177,11995
232926,3250
156037,8950
279956,4288
152000,7950
138122,7495
24000,28500
31893,33991


In [14]:
#let's dump to pandas to calculate the correlation
pandas_df= df\
    .select(get_odo('odometer').alias('odometer'), convert_value('price').alias('price'),)\
    .where(col('odometer').isNotNull() & (col('odometer') >=1000))\
    .toPandas()

In [15]:
pandas_df.head()

,odometer,price
0,140534,16995.0
1,13666,31881.0
2,291177,11995.0
3,232926,3250.0
4,156037,8950.0


In [16]:
pandas_df.corr(method='pearson')

,odometer,price
odometer,1.000000,-0.727035
price,-0.727035,1.000000


In [17]:
pandas_df_ = df\
    .select(get_year('Url').alias('year'), convert_value('price').alias('price'),)\
    .where(col('year')>=1950 )\
    .groupBy('year')\
    .agg(avg('price').cast('integer'))\
    .toPandas()

In [18]:
pandas_df_.corr(method='pearson')

,year,CAST(avg(price) AS INT)
year,1.000000,0.508624
CAST(avg(price) AS INT),0.508624,1.000000


In [19]:
# Based on the analysis of 500 toyota used cars, odometer has stronger correlation to price. 
# In other words, odometer reading is more important for used cars. 